# Riot API로 받지 못하는 정보 OP.GG에서 크롤링하기

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

### 각 라인의 탑 티어 챔피언 구하기

In [2]:
url = 'https://www.op.gg/champion/statistics'
html = requests.get(url).text
soup = BeautifulSoup(html, 'html.parser')

- 필요한 함수들

In [3]:
# 챔피언, 아이템, 스펠, 룬 데이터
champion_df = pd.read_csv('../00. data/champions.csv', index_col = 0)
champion_df['lower_name'] = champion_df['eng_name'].apply(lambda r : r.lower()) # OP.GG의 챔피언 찾기를 위해 소문자화
item_df = pd.read_csv('../00. data/items.csv', index_col = 0)
spell_df = pd.read_csv('../00. data/spell.csv', index_col = 0)
rune_df = pd.read_csv('../00. data/runes.csv', index_col = 0)

In [4]:
# 챔피언의 영문이름을 한글로 바꿔주는 함수
def translation_champion(eng):
    try :
        if eng == 'Wukong' :
            champion = '오공'
        elif eng == 'Nunu &amp; Willump':
            champion = '누누와 월럼프'
        else :
            eng = eng.lower().replace(' ','').replace('.','')
            champion = champion_df['name'][champion_df['name'][champion_df['lower_name'] == eng].index[0]]
    except:
        champion = eng 
    return champion

In [5]:
# 아이템키로 아이템 이름 불러오는 함수
def searchItem(itemKey):
    item = item_df['name'][item_df['name'][item_df['key'] == itemKey].index[0]]
    return item 

In [6]:
# 영어이름의 스펠을 한글로 바꿔주는 함수
def translation_spell(eng):
    spell = spell_df['name'][spell_df['name'][spell_df['eng'] == eng].index[0]]
    return spell

In [7]:
# 룬키로 룬 이름 불러오는 함수
def searchRune(runeKey):
    rune = rune_df['name'][rune_df['name'][rune_df['key'] == runeKey].index[0]]
    return rune

In [8]:
# 태그 지우는 함수
def removeTags(strings):
    strings = re.sub('<.+?>', '', str(strings), 0).strip().replace('[','').replace(']','')
    return strings

In [9]:
# OP.GG 아이템이미지의 키를 구하는 함수 + 룬
def findItem(itemLink):
    items = re.findall(r'\d\d\d\d.png', str(itemLink))
    items = [item.strip('.png') for item in items]
    
    return items

In [10]:
# OP.GG 스킬이미지의 영문 이름을 구하는 함수
def findSkill(link):
    name = re.findall(r'spell/.*png', str(link))
    name = ''.join(name).strip('spell/').strip('.png')
    return name

In [11]:
lanes = ['TOP', 'JUNGLE', 'MID', 'ADC', 'SUPPORT']

In [12]:
for lane in lanes:
    top10 = soup.find(attrs={'class':f'tabItem champion-trend-tier-{lane}'}).select('.champion-index-table__name')[:10] # 10개만 가져오기
    top10 = removeTags(top10).replace("'",'')
    top10 = top10.split(', ')
    kor_champions = [translation_champion(champ) for champ in top10]
    kor_champions = ', '.join(kor_champions)
    print(f'{lane}의 최고티어 챔피언:',kor_champions)

TOP의 최고티어 챔피언: 탐 켄치, 카밀, 피오라, 세트, 이렐리아, 잭스, 릴리아, 말파이트, 아트록스, 뽀삐
JUNGLE의 최고티어 챔피언: 리 신, 자르반 4세, 니달리, 에코, 카서스, 킨드레드, 렉사이, 샤코, 그레이브즈, 엘리스
MID의 최고티어 챔피언: 탈론, 카타리나, 아크샨, 제드, 르블랑, 애니비아, 야스오, 판테온, 카시오페아, 아우렐리온 솔
ADC의 최고티어 챔피언: 애쉬, 직스, 베인, 사미라, 이즈리얼, 스웨인, 진, 아펠리오스, 루시안, 카시오페아
SUPPORT의 최고티어 챔피언: 레오나, 블리츠크랭크, 쓰레쉬, 룰루, 마오카이, 노틸러스, 세나, 파이크, 질리언, 렐


### 원하는 챔피언의 기록 가져오기

In [13]:
# 샘플은 탑라인의 이렐리아
lane = 'top'
champion ='irelia'
url = f'https://www.op.gg/champion/{champion}/statistics/{lane}'
html = requests.get(url).text
soup = BeautifulSoup(html, 'html.parser')

In [14]:
win_rank = soup.select('.champion-stats-trend-rank')[0].find('b').text + soup.select('.champion-stats-trend-rank')[0].find('span').text
pick_rank = soup.select('.champion-stats-trend-rank')[1].find('b').text + soup.select('.champion-stats-trend-rank')[1].find('span').text
print('승률 순위:', win_rank, '픽률 순위:', pick_rank)

승률 순위: 42/56 픽률 순위: 2/56


### 원하는 챔피언의 추천 세팅 

- 스펠 , 스킬트리, 아이템 트리

In [15]:
for i in range(2):
    tmp = soup.select('ul.champion-stats__list')[i].find_all('img')[0] 
    tmp2 = soup.select('ul.champion-stats__list')[i].find_all('img')[1]
    pickRate = soup.select('.champion-overview__stats.champion-overview__stats--pick')[i].find('strong').text
    pickTotal = soup.select('.champion-overview__stats.champion-overview__stats--pick')[i].find('span').text
    winRate = soup.select('.champion-overview__stats.champion-overview__stats--win')[i].find('strong').text
    print('추천 스펠 : '+ translation_spell(findSkill(tmp)), translation_spell(findSkill(tmp2)), ' | 픽률 : '+ pickRate, '총표본 수 : ' + pickTotal, '승률 : '+ winRate)
skillSeq = []
for i in range(3):
    skill = findSkill(soup.select('ul.champion-stats__list')[2].find_all('img')[i*2])
    skillSeq.append(skill.lower().split(champion)[1])
pickRate = soup.select('.champion-overview__stats.champion-overview__stats--pick')[2].find('strong').text
pickTotal = soup.select('.champion-overview__stats.champion-overview__stats--pick')[2].find('span').text
winRate = soup.select('.champion-overview__stats.champion-overview__stats--win')[2].find('strong').text
print('스킬 트리: ',' -> '.join(skillSeq).upper(),' | 픽률:', pickRate, '총표본 수 : ' + pickTotal, '승률:', winRate)    
for i in range(10):
    tmp = soup.select('td.champion-overview__data.champion-overview__border.champion-overview__border--first')[i].find_all('img')
    keys = findItem(tmp)
    items = [searchItem(int(key)) for key in keys]
    items = ', '.join(items)
    pickRate = soup.select('.champion-overview__stats.champion-overview__stats--pick.champion-overview__border')[i].find('strong').text
    pickTotal = soup.select('.champion-overview__stats.champion-overview__stats--pick.champion-overview__border')[i].find('span').text
    winRate = soup.select('.champion-overview__stats.champion-overview__stats--win.champion-overview__border')[i].find('strong').text
    if i < 2:
        msg = '시작아이템 :'
    elif i >= 2 and i < 7:
        msg = '추천 빌드 :'
    else:
        msg = '신발 :'
    print(msg + items, '| 픽률 : '+ pickRate,'총표본 수 : ' + pickTotal, '승률 : '+ winRate)

추천 스펠 : 점멸 순간이동  | 픽률 : 77.71% 총표본 수 : 23,139 승률 : 49.05%
추천 스펠 : 점멸 점화  | 픽률 : 20.79% 총표본 수 : 6,190 승률 : 49.48%
스킬 트리:  Q -> W -> E  | 픽률: 38.99% 총표본 수 : 5,187 승률: 63.2%
시작아이템 :도란의 검, 체력 물약 | 픽률 : 81.04% 총표본 수 : 23,568 승률 : 49.69%
시작아이템 :부패 물약 | 픽률 : 10.49% 총표본 수 : 3,050 승률 : 46.23%
추천 빌드 :몰락한 왕의 검, 선혈포식자, 스테락의 도전 | 픽률 : 24.13% 총표본 수 : 3,016 승률 : 64.79%
추천 빌드 :몰락한 왕의 검, 불멸의 철갑궁, 구인수의 격노검 | 픽률 : 14.54% 총표본 수 : 1,818 승률 : 61.94%
추천 빌드 :몰락한 왕의 검, 불멸의 철갑궁, 마법사의 최후 | 픽률 : 9.41% 총표본 수 : 1,176 승률 : 62.07%
추천 빌드 :몰락한 왕의 검, 신성한 파괴자, 스테락의 도전 | 픽률 : 6.74% 총표본 수 : 842 승률 : 62%
추천 빌드 :몰락한 왕의 검, 불멸의 철갑궁, 죽음의 무도 | 픽률 : 6.58% 총표본 수 : 823 승률 : 62.94%
신발 :판금 장화 | 픽률 : 67.21% 총표본 수 : 17,926 승률 : 50.91%
신발 :헤르메스의 발걸음 | 픽률 : 29.43% 총표본 수 : 7,849 승률 : 53.34%
신발 :명석함의 아이오니아 장화 | 픽률 : 2.57% 총표본 수 : 685 승률 : 62.77%


- 룬

In [16]:
mainRune = {'Precision' : '정밀', 'Domination' : '지배', 'Sorcery' : '마법', 'Resolve' : '결의', 'Inspiration' : '영감'}

In [18]:
# 첫번째 추천룬
total_info = soup.select_one('.tabItem.ChampionKeystoneRune-1')
rate_info = total_info.select('.champion-overview__stats.champion-overview__stats--pick')
rec_runes = soup.select('.champion-stats-summary-rune__name')
rune_info = soup.select('.champion-stats-summary-rune__rate')
eng_rune = rec_runes[0].text.split(' + ')
kor_rune = mainRune[eng_rune[0]] + ' + ' + mainRune[eng_rune[1]]
pickRate = rune_info[0].find('strong').text
winRate = rune_info[0].find_all('span')[-1].text
print(kor_rune, '| 픽률 : ' + pickRate, '승률 : ' + winRate)
for i in range(2): 
    pickRate = rate_info[i].find('strong').text
    winRate= rate_info[i].find_all('strong')[-1].text
    pickTotal = rate_info[i].find_all('span')[1].text
    print('추천룬 | 픽률 : ' + pickRate, '승률 : ' + winRate, '총 표본수 : ' + pickTotal)
    for k in range(33):
        if i == 1:
            k += 33
        link = total_info.find_all('img')[k].attrs['src']
        if 'e_grayscale' in str(link):
            pass
        else:
            print(searchRune(int(''.join(findItem(link)))))

정밀 + 결의 | 픽률 : 78.45% 승률 : 49.12%
추천룬 | 픽률 : 25.84% 승률 : 48.45% 총 표본수 : 7,695
정밀
정복자
승전보
전설: 민첩함
최후의 저항
결의
뼈 방패
소생
공격속도 + 10%
적응형 능력치 + 9
방어력 + 9
추천룬 | 픽률 : 9.9% 승률 : 49.71% 총 표본수 : 2,947
정밀
정복자
승전보
전설: 민첩함
최후의 저항
결의
뼈 방패
불굴의 의지
공격속도 + 10%
적응형 능력치 + 9
방어력 + 9


In [19]:
# 두번째 추천 룬
rate_info = soup.select_one('.ChampionKeystoneRune-2').select('.champion-overview__stats.champion-overview__stats--pick')
eng_rune = rec_runes[1].text.split(' + ')
kor_rune = mainRune[eng_rune[0]] + ' + ' + mainRune[eng_rune[1]]
pickRate = rune_info[1].find('strong').text
winRate = rune_info[1].find_all('span')[-1].text
print(kor_rune, '| 픽률 : ' + pickRate, '승률 : ' + winRate)
for i in range(2):
    pickRate = rate_info[i].find('strong').text
    winRate= rate_info[i].find_all('strong')[-1].text
    pickTotal = rate_info[i].find_all('span')[1].text
    print('추천룬 | 픽률 : ' + pickRate, '승률 : ' + winRate, '총 표본수 : ' + pickTotal)
    for k in range(33):
        if i == 1:
                k += 33
        link = soup.select_one('.ChampionKeystoneRune-2').find_all('img')[k].attrs['src']
        if 'e_grayscale' in str(link):
            pass
        else:
            print(searchRune(int(''.join(findItem(link)))))

정밀 + 영감 | 픽률 : 15.10% 승률 : 49.17%
추천룬 | 픽률 : 2.9% 승률 : 48.67% 총 표본수 : 865
정밀
정복자
승전보
전설: 민첩함
최후의 저항
영감
비스킷 배달
시간 왜곡 물약
공격속도 + 10%
적응형 능력치 + 9
방어력 + 9
추천룬 | 픽률 : 1.46% 승률 : 50.46% 총 표본수 : 436
정밀
정복자
승전보
전설: 민첩함
최후의 저항
영감
비스킷 배달
시간 왜곡 물약
공격속도 + 10%
적응형 능력치 + 9
마법저항력 + 8


### 매치업

In [19]:
url = f'https://www.op.gg/champion/{champion}/statistics/{lane}/matchup'
html = requests.get(url).text
soup = BeautifulSoup(html, 'html.parser')

In [20]:
for i in range(50):
    champ = soup.select('.champion-matchup-list__champion')[i].find('span').text
    winRate = soup.select('.champion-matchup-list__champion')[i].find_all('span')[1].string
    winRate = [item.strip() for item in winRate if str(item)]
    winRate = ''.join(winRate)
    play_rate = soup.select('.champion-matchup-list__totalplayed')[i].find('span').text
    play_num = soup.select('.champion-matchup-list__totalplayed')[i].find('small').text
    print(f'{translation_champion(champion)} vs {translation_champion(champ)} 상대승률 :',winRate)
    print(f'통계량: {play_num} 통계 비율 {play_rate}')

이렐리아 vs 피오라 상대승률 : 47.94%
통계량: 20,985 통계 비율 7.91%
이렐리아 vs 세트 상대승률 : 48.15%
통계량: 20,320 통계 비율 7.66%
이렐리아 vs 카밀 상대승률 : 49.5%
통계량: 12,495 통계 비율 4.71%
이렐리아 vs 레넥톤 상대승률 : 50.96%
통계량: 9,866 통계 비율 3.72%
이렐리아 vs 잭스 상대승률 : 47.42%
통계량: 9,806 통계 비율 3.70%
이렐리아 vs 말파이트 상대승률 : 50.5%
통계량: 9,726 통계 비율 3.67%
이렐리아 vs 제이스 상대승률 : 56.3%
통계량: 8,993 통계 비율 3.39%
이렐리아 vs 사일러스 상대승률 : 52.77%
통계량: 8,525 통계 비율 3.21%
이렐리아 vs 탐 켄치 상대승률 : 48.92%
통계량: 8,265 통계 비율 3.12%
이렐리아 vs 다리우스 상대승률 : 52.49%
통계량: 7,623 통계 비율 2.87%
이렐리아 vs 가렌 상대승률 : 48.6%
통계량: 7,002 통계 비율 2.64%
이렐리아 vs 리븐 상대승률 : 47.33%
통계량: 6,861 통계 비율 2.59%
이렐리아 vs 아트록스 상대승률 : 52.81%
통계량: 6,771 통계 비율 2.55%
이렐리아 vs 오른 상대승률 : 49.74%
통계량: 6,620 통계 비율 2.50%
이렐리아 vs 트런들 상대승률 : 49.07%
통계량: 6,054 통계 비율 2.28%
이렐리아 vs 케넨 상대승률 : 53.34%
통계량: 5,924 통계 비율 2.23%
이렐리아 vs 베인 상대승률 : 50.97%
통계량: 5,038 통계 비율 1.90%
이렐리아 vs 쉔 상대승률 : 48.82%
통계량: 4,842 통계 비율 1.83%
이렐리아 vs 모데카이저 상대승률 : 50.59%
통계량: 4,675 통계 비율 1.76%
이렐리아 vs 티모 상대승률 : 52.4%
통계량: 4,477 통계 비율 1.69%
이렐리아 vs 오공 상대승률 : 47.59%
통